In [13]:
import requests
from datetime import datetime
import json

In [50]:
Series = {"MGSX" : {"Dataset_ID" : "LMS", "Name": "Unemployment"},
"KAC3" : {"Dataset_ID" : "LMS", "Name": "Wage Growth"},
"YBUY" : {"Dataset_ID" : "LMS", "Name": "Full-Time Hours"},
"YBVB" : {"Dataset_ID" : "LMS", "Name": "Part-Time Hours"},
"LZVB" : {"Dataset_ID" : "PRDY", "Name": "Productivity - Hourly"},
"A4YM" : {"Dataset_ID" : "PRDY", "Name": "Productivity - Worker"},
"BEIR" : {"Dataset_ID" : "LMS", "Name": "Redundancy rate"},
"BCJE" : {"Dataset_ID" : "UNEM", "Name": "Benefits Claimant Count"},
"G7G3" : {"Dataset_ID" : "PSE", "Name": "Public Sector Employment"}}


In [51]:
for Series_ID in Series.copy():
    url = f'https://api.ons.gov.uk/timeseries/{Series_ID}/dataset/{Series[Series_ID]["Dataset_ID"]}/data'
    Series[Series_ID]['request'] = requests.get(url)

In [33]:
req = Series['LZVB']['request']
data = json.loads(req.text)
processed_data = []
for entry in data['quarters']:
    processed_data.append({
            'Date' : entry['date'],
            'Value' : entry['value']
    })


In [5]:
with open("test.json", "w") as f:
    json.dump(processed_data, f)

In [48]:
data['description']['title']

'Public Sector Employment by Sector; Total Public Sector; UK; FTE; SA;Thousands'

In [14]:
for entry in data['months']:
    print(entry['date'])

a helper function for converting quarters:

In [40]:
def quarterly_to_iso(quarterly_string):
  # Split the string into year and quarter parts
  year, quarter = quarterly_string.split(' Q')
  # Calculate the month corresponding to the quarter
  month = (int(quarter) - 1) * 3 + 1
  # Format the date as an ISO-compliant string
  iso_string = f"{year}-{month:02d}-01T00:00:00"
  return iso_string

In [27]:
quarterly_to_iso('2001 Q4')

'2001-10-01T00:00:00'

In [29]:
l = [9,8,8]
for i,li in enumerate(l):
    print(i)

0
1
2


In [60]:
for i,Series_ID in enumerate(Series):
    req = Series[Series_ID]['request']
    data = json.loads(req.text)
    processed_data = []
    if len(data['months'])>0: #Our Monthly Data
        for entry in data['months']:
            processed_data.append({
                'Date' : datetime.strptime(entry['date'], '%Y %b').isoformat(),
                'Value' : entry['value']
            })
    elif(len(data['quarters'])>0): #Our Quarterly Data
        for entry in data['quarters']:
            processed_data.append({
                'Date' : quarterly_to_iso(entry['date']),
                'Value' : entry['value']
            })
    with open(f'data/onsFig{i+1}_data.json', 'w') as f:
        json.dump(processed_data, f)

    with open(f'onsFig{i+1}.json', 'w') as f:
        f.write(f"""{{ "$schema": "https://vega.github.io/schema/vega-lite/v4.json", "description": "A line chart showing {data['description']['title']}.", "data": {{ "url": "week5/data/onsFig{i+1}_data.json" }}, "transform": [ {{     "filter": {{     "field": "Date",     "timeUnit": "yearmonth",     "gt": "2010-01"     }} }} ], "mark": {{"type": "line", "tooltip":true, "point": {{"size":5}}}}, "encoding": {{ "x": {{     "field": "Date",     "type": "temporal",     "axis": {{"title": null}} }}, "y": {{"field": "Value", "type": "quantitative", "axis":{{"title":null}}}} }}, "title": {{     "text": "{Series[Series_ID]['Name']}",     "align": "left", "subtitleFontSize": 1,    "subtitle": "{data['description']['title']}",     "anchor": "start"     }}, "width": "container", "height": 250}} """)

